In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler,LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
data = pd.read_csv('C:\\Users\\DELL\\Downloads\\machineWithPython\\newYorkCityAirbnb\\AB_NYC_2019.csv')
data.drop(columns=['name','last_review','id','host_id','reviews_per_month'],inplace=True)
c = ['latitude','longitude','price','minimum_nights','number_of_reviews','calculated_host_listings_count']
for i in c:
    data[i] = np.log1p(data[i])

C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [2]:
data['host_name'] = data['host_name'].fillna(data['host_name'].mode())
label = LabelEncoder()

data['neighbourhood'] = label.fit_transform(data['neighbourhood'])
data['host_name'] = label.fit_transform(data['host_name'])
encode = ['neighbourhood_group','room_type']
data = pd.get_dummies(data, columns=encode,dtype='int')
data.drop(columns='longitude',inplace=True)
skewness = data.select_dtypes(include=['float64', 'int64']).apply(lambda x: x.skew())
print("Skewness of numerical columns:\n", skewness)
highly_skewed = [
    "calculated_host_listings_count", "neighbourhood_group_Bronx", 
    "neighbourhood_group_Queens", "neighbourhood_group_Staten Island", 
    "room_type_Shared room"
]

for col in highly_skewed:
    data[col] = np.log1p(data[col])

new_skewness = data[highly_skewed].apply(lambda x: x.skew())
print("Skewness after transformation:\n", new_skewness)

Skewness of numerical columns:
 host_name                             0.057993
neighbourhood                         0.256005
latitude                              0.233064
price                                 0.553105
minimum_nights                        1.498510
number_of_reviews                     0.362924
calculated_host_listings_count        3.279837
availability_365                      0.763408
neighbourhood_group_Bronx             6.468545
neighbourhood_group_Brooklyn          0.361088
neighbourhood_group_Manhattan         0.229460
neighbourhood_group_Queens            2.400202
neighbourhood_group_Staten Island    11.318186
room_type_Entire home/apt            -0.078722
room_type_Private room                0.174218
room_type_Shared room                 6.259199
dtype: float64
Skewness after transformation:
 calculated_host_listings_count        2.275767
neighbourhood_group_Bronx             6.468545
neighbourhood_group_Queens            2.400202
neighbourhood_group_Staten I

In [3]:
x = data.drop(columns='price',axis=1)
y = data[['price']]

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x,y,shuffle=True,test_size=0.3,random_state=42)

In [5]:
x_scalar = MinMaxScaler()
y_scalar = MinMaxScaler()

x_train_scalar = x_scalar.fit_transform(x_train)
x_test_scalar = x_scalar.transform(x_test)

y_train_scalar = y_scalar.fit_transform(y_train)
y_test_scalar = y_scalar.transform(y_test)

In [6]:
knn_regressor = KNeighborsRegressor(n_neighbors=19)
knn_regressor.fit(x_train_scalar, y_train_scalar)

KNeighborsRegressor(n_neighbors=19)

In [7]:
y_pred = knn_regressor.predict(x_test_scalar)


In [8]:
train_preds = knn_regressor.predict(x_train_scalar)
test_preds = knn_regressor.predict(x_test_scalar)


train_mse = mean_squared_error(y_train_scalar, train_preds, squared=False)
test_mse = mean_squared_error(y_test_scalar, test_preds, squared=False)
print(f"Training MSE: {train_mse}")
print(f"Test MSE: {test_mse}")

print("\n=======================================\n")


train_mae = mean_absolute_error(y_train_scalar, train_preds)
test_mae = mean_absolute_error(y_test_scalar, test_preds)
print(f"Training MAE: {train_mae}")
print(f"Test MAE: {test_mae}")


print("\n=======================================\n")


train_r2 = r2_score(y_train_scalar, train_preds)
test_r2 = r2_score(y_test_scalar, test_preds)
print(f"Training R-squared: {train_r2}")
print(f"Test R-squared: {test_r2}")

Training MSE: 0.048885765783163136
Test MSE: 0.050178343429363816


Training MAE: 0.03474133003290804
Test MAE: 0.03641913163378019


Training R-squared: 0.5846803543984251
Test R-squared: 0.5480662176340311


C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [9]:
knn = KNeighborsRegressor()

param_grid = {'n_neighbors': range(1, 31)}

grid_search = GridSearchCV(knn, param_grid, cv=10)

grid_search.fit(x_train_scalar, y_train_scalar)


C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': range(1, 31)})

In [10]:
best_k = grid_search.best_params_['n_neighbors']
best_score = grid_search.best_score_
print(f"Best K: {best_k}")
print(f"Best Cross-Validation : {best_score:.2f}")

Best K: 23
Best Cross-Validation : 0.54


In [11]:
best_knn = KNeighborsRegressor(n_neighbors=best_k)
best_knn.fit(x_train_scalar, y_train_scalar)

KNeighborsRegressor(n_neighbors=23)

In [12]:
y_pred = best_knn.predict(x_test_scalar)


In [13]:
accuracy = r2_score(y_test_scalar, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.55
